<a href="https://colab.research.google.com/github/malofeevakate/app_taxi_research/blob/main/taxi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
sns.set(rc={'figure.figsize':(12,6)}, style="whitegrid")

In [ ]:
# загружаем данные
df = pd.read_csv('churn.csv')

In [ ]:
df.head()

,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,city,last_trip_date,phone,signup_date,surge_pct,trips_in_first_30_days,luxury_car_user,weekday_pct
0,3.67,5.0,4.7,1.10,King's Landing,2014-06-17,iPhone,2014-01-25,15.4,4,True,46.2
1,8.26,5.0,5.0,1.00,Astapor,2014-05-05,Android,2014-01-29,0.0,0,False,50.0
2,0.77,5.0,4.3,1.00,Astapor,2014-01-07,iPhone,2014-01-06,0.0,3,False,100.0
3,2.36,4.9,4.6,1.14,King's Landing,2014-06-29,iPhone,2014-01-10,20.0,9,True,80.0
4,3.13,4.9,4.4,1.19,Winterfell,2014-03-15,Android,2014-01-27,11.8,14,False,82.4


In [ ]:
# имеются пропущенные данные в рейтингах водителей и телефонах
# даты формата object
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   avg_dist                50000 non-null  float64
 1   avg_rating_by_driver    49799 non-null  float64
 2   avg_rating_of_driver    41878 non-null  float64
 3   avg_surge               50000 non-null  float64
 4   city                    50000 non-null  object 
 5   last_trip_date          50000 non-null  object 
 6   phone                   49604 non-null  object 
 7   signup_date             50000 non-null  object 
 8   surge_pct               50000 non-null  float64
 9   trips_in_first_30_days  50000 non-null  int64  
 10  luxury_car_user         50000 non-null  bool   
 11  weekday_pct             50000 non-null  float64
dtypes: bool(1), float64(6), int64(1), object(4)
memory usage: 4.2+ MB


In [ ]:
# Изменяем тип для дат
df.last_trip_date = pd.to_datetime(df.last_trip_date)
df.signup_date = pd.to_datetime(df.signup_date)

In [ ]:
# описательные статистики количественных переменных
df.describe()

,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,surge_pct,trips_in_first_30_days,weekday_pct
count,50000.000000,49799.000000,41878.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,5.796827,4.778158,4.601559,1.074764,8.849536,2.278200,60.926084
std,5.707357,0.446652,0.617338,0.222336,19.958811,3.792684,37.081503
min,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,2.420000,4.700000,4.300000,1.000000,0.000000,0.000000,33.300000
50%,3.880000,5.000000,4.900000,1.000000,0.000000,1.000000,66.700000
75%,6.940000,5.000000,5.000000,1.050000,8.600000,3.000000,100.000000
max,160.960000,5.000000,5.000000,8.000000,100.000000,125.000000,100.000000


In [ ]:
# характеристики временных фичей
df.describe(include='datetime')

C:\Users\User\AppData\Local\Temp\ipykernel_8884\1397772673.py:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe(include='datetime')
C:\Users\User\AppData\Local\Temp\ipykernel_8884\1397772673.py:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe(include='datetime')


,last_trip_date,signup_date
count,50000,50000
unique,182,31
top,2014-06-29 00:00:00,2014-01-18 00:00:00
freq,2036,2948
first,2014-01-01 00:00:00,2014-01-01 00:00:00
last,2014-07-01 00:00:00,2014-01-31 00:00:00


Начнем решать поставленную задачу. Предположим, что юзер неактивен, если не был в приложении (не был активен) последние 30 дней.  
Создаем лейбл churn/not churn

In [ ]:
# возьмем последнюю дату поездки в датасете за дату расчета
df.last_trip_date.max()

Timestamp('2014-07-01 00:00:00')

In [ ]:
# определим период неактивности для каждого юзера
df['days_since_last_trip'] = df.last_trip_date.max() - df.last_trip_date

In [ ]:
df.days_since_last_trip

0        14 days
1        57 days
2       175 days
3         2 days
4       108 days
          ...   
49995    26 days
49996   157 days
49997    40 days
49998   167 days
49999    72 days
Name: days_since_last_trip, Length: 50000, dtype: timedelta64[ns]

In [ ]:
# Преобразуем в int:
df['days_since_last_trip'] = df['days_since_last_trip'].dt.days

In [ ]:
# создаем лейбл
df['churn'] = df.days_since_last_trip.apply(lambda x: 'churn' if x > 30 else 'not_churn')
df[['days_since_last_trip', 'churn']]

,days_since_last_trip,churn
0,14,not_churn
1,57,churn
2,175,churn
3,2,not_churn
4,108,churn
...,...,...
49995,26,not_churn
49996,157,churn
49997,40,churn
49998,167,churn


In [ ]:
# почти 2/3 юзеров не использовали сервис в последнем месяце, почему?
df.churn.value_counts(normalize=True).mul(100)

churn        62.392
not_churn    37.608
Name: churn, dtype: float64

In [ ]:
# может, виновата платформа?
pd.crosstab(df.churn, df.phone)

phone,Android,iPhone
churn,,
churn,11876,19057
not_churn,3146,15525


Как будто мобильная платформа влияет на то, пользовался ли юзер приложением последний месяц. Проверим нашу гипотезу с помощью статистического теста (поскольку есть две категориальные переменные, нужен хи-квадрат):

- $H_0$: взаимосвязи между переменными нет
- $H_1$: взаимосвязь есть

In [ ]:
from scipy.stats import chi2_contingency, chi2
stat, p, dof, expected = chi2_contingency(pd.crosstab(df.churn, df.phone))
stat, p

(2558.394212267425, 0.0)

In [ ]:
# Интерпретируем результат:
prob = 0.95
alpha = 1.0 - prob
if p <= alpha:
    print('Отклоняем H0')
else:
    print('Не отклоняем H0')

Отклоняем H0


Итак, похоже что нужен дополнительный анализ работы приложения на андроиде, поскольку значительная часть ушедших юзеров именно с андроида

In [ ]:
# окей, что насчет поведения юзеров из разных городов?
pd.crosstab(df.churn, df.city)

city,Astapor,King's Landing,Winterfell
churn,,,
churn,12306,3767,15123
not_churn,4228,6363,8213


Кажется, что юзеры из разных городов по-разному активничают последний месяц. Проверим нашу гипотезу с помощью статистического теста (поскольку есть две категориальные переменные, нужен хи-квадрат):

- $H_0$: взаимосвязи между переменными нет
- $H_1$: взаимосвязь есть

In [ ]:
from scipy.stats import chi2_contingency, chi2
stat, p, dof, expected = chi2_contingency(pd.crosstab(df.churn, df.city))
stat, p

(3821.5510225559633, 0.0)

In [ ]:
# Интерпретируем результат:
prob = 0.95
alpha = 1.0 - prob
if p <= alpha:
    print('Отклоняем H0')
else:
    print('Не отклоняем H0')

Отклоняем H0


Окей, получается, что юзеры в последний месяц в разных городах ведут себя по-разному. Это касается только указанного периода, или же это особенность поведения пользователей? Сравним к примеру число поездок в первый месяц по разным городам

In [ ]:
# прежде всего проверим данные по городам на нормальность
from scipy.stats import shapiro

In [ ]:
df_w = df.query('city == "Winterfell"').trips_in_first_30_days
df_a = df.query('city == "Astapor"').trips_in_first_30_days
df_kl = df.query('city == "King\'s Landing"').trips_in_first_30_days

In [ ]:
stats.shapiro(df_w)

C:\Users\User\anaconda3\lib\site-packages\scipy\stats\_morestats.py:1761: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


ShapiroResult(statistic=0.593726634979248, pvalue=0.0)

In [ ]:
stats.shapiro(df_a)

ShapiroResult(statistic=0.5382282733917236, pvalue=0.0)

In [ ]:
stats.shapiro(df_kl)

ShapiroResult(statistic=0.550529956817627, pvalue=0.0)

Распределения не являются нормальными, поэтому используем непараметрический аналог ANOVA – критерий Краскела-Уоллиса

In [ ]:
stats.kruskal(df_w, df_a, df_kl)

KruskalResult(statistic=221.32105325320535, pvalue=8.724567791804361e-49)

В активности в первые 30 дней с момента регистрации между водителями ***из разных городов*** обнаружены  статистически значимые различия.  
Тогда влияет ли активность в первые 30 дней на последующую активность, например в последние 30 дней?  
Поскольку распределение переменной trips_in_first_30_days не является нормальным, сравним активности оставшихся и ушедших юзеров с помощью критерия Манна - Уитни

In [ ]:
from scipy.stats import mannwhitneyu

In [ ]:
mannwhitneyu(df.query('churn == "churn"').trips_in_first_30_days, df.query('churn == "not_churn"').trips_in_first_30_days)

MannwhitneyuResult(statistic=234767452.0, pvalue=0.0)

In [ ]:
df.groupby('churn', as_index = False).trips_in_first_30_days.mean()

,churn,trips_in_first_30_days
0,churn,1.658482
1,not_churn,3.306318


Выводы:  
Статистически значимые различия в поведении юзеров наблюдаются в разрезе платформ, городов и первоначальной активности, поэтому:  
1. Необходимо рассмотреть особенности работы приложения на платформе андроид, возможны баги  
2. Нужен дополнительный анализ работы приложения и рынков такси в двух городах из трех (возможно, зашел более выгодный для юзеров конкурент?).
3. Видно, что ушедшие юзеры были изначально менее активны. Возможно стоит рассмотреть вариант улучшения программы лояльности и бонусов для водителей, или улучшить таргетинг  